The purpose of this script is to,

1. Read all the files(7 years of data and each file contains 3 months data)
2. Delete unwanted records from each file.
3. Drop unwated columns.
4. Merge the files to have all the records in a single file.
5. Remove duplicates and retain only the latest one.
6. Perfom the final formatting

In [ ]:
import glob
import pandas as pd
import datetime
import os

# get all the file names in the below directory.. 

path =r'file path'
filenames = glob.glob(path + "/*.xlsx")

# set the header for the dataframe
dfs = pd.DataFrame(columns=["Agency","Candidate_ID","Title","First_Name","Middle_Name","Last_Name","Email",
                            "Principal_Department",
                            "Classification","Voluntary_Red_Pkg","Defence_reservist_service","Address_Line1",
                            "Address_Line2","City","Zip","App_Crt_Date",
                            "Internal_Flag","Mobile_Phn","Personnel_ID",
                            "DOB","Gender","Employed_with_DOE","Employed_School","Profile_Crt_Date","User_name"])


#Read the excel files one by one, clean the file and merge into single one
for filename in filenames:
    tmp = pd.read_excel(filename)
    #Remove first three lines
    candidate = tmp[2:]
    #drop unwanted columns
    candidate = candidate.drop(candidate.columns[[0,1,2,3,4,6,7,8,9,10,11,12,28,30,31,40,41]], axis=1) 
    candidate.columns = dfs.columns
    dfs = pd.concat([dfs,candidate])
    

#Sort dataframe based on cadidate id and application date
dfs.sort_values(["Candidate_ID","App_Crt_Date"], ascending=False,inplace = True)
#Remove duplicate candidate records and retain the one with lastest application date
dfs.drop_duplicates(subset ="Candidate_ID", 
                     keep = 'first', inplace = True)

cand = dfs

dfs['Address_Line1'] = dfs.Address_Line1.fillna('')
dfs['Address_Line2'] = dfs.Address_Line2.fillna('')
dfs['Address_Line2'] = dfs['Address_Line2'].astype('str')

dfs["Principal_Department"] = dfs["Agency"]
dfs = dfs.drop(dfs.columns[[0]], axis=1) 


# Reorder dataframe
candidatefinal = dfs[["Candidate_ID","First_Name","Last_Name","Middle_Name","Email","Principal_Department",
                            "Classification","Voluntary_Red_Pkg","Defence_reservist_service","Address_Line1",
                            "Address_Line2","City","Zip","App_Crt_Date",
                            "Internal_Flag","Mobile_Phn","Title", "DOB","Gender","Personnel_ID",
                     "Employed_with_DOE","Employed_School","Profile_Crt_Date","User_name"]]

#Final formatting
candidatefinal['App_Crt_Date'] = candidatefinal['App_Crt_Date'].dt.date

# Add future column names with spacesand assign record type

candidatefinal.insert(19, 'Application_Count', '')
candidatefinal.insert(20, 'DOE_Department', '')
candidatefinal.insert(0, 'Record_Type', 'CORP')



#add timestamp to the filename 
filetime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
filename = 'CORP_Candidate' + filetime+'.csv'
print(filename)


outpath = os.path.join(r'file path', filename)
         
export_csv = candidatefinal.to_csv (outpath, index = None, header=True)
         